In [1]:
from flask import Flask, request, jsonify
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

# 타로카드 라우트
@app.route("/api/interpret",methods=["POST"])

def tarot_card():
    data = request.json
    cards = data.get("cards")
    reason = data.get("reason")
    user_input = data.get("user_input")
    user_select = data.get("user_select")
    print("recived data: ",data)
    if not cards or len(cards) != 3:
        return jsonify({"error": "카드를 다시 선택해 주세요."}),400
    
    class TarotBot:
        def __init__(self, api_key_path):
            #OpenAI API 설정
            self.api_key = self.load_api_key(api_key_path)
            self.model = ChatOpenAI(openai_api_key = self.api_key, model="gpt-4o")

        def load_api_key(self, file_path):
            with open(file_path, 'r') as file:
                api_key = file.read().strip()
                return api_key
            
        # 사용자 고민이 있을때 타로 해석
        def interpret_cards(self, user_input, cards):
            try:
                tarot_template_reason = PromptTemplate(
                    template=(
                        "사용자의 질문: '{user_input}'\n"
                        "타로카드: {cards}\n"
                        "{cards}의 의미는 한줄로 간략히 출력하고, {user_input}를 주제로 종합한 해석은 최대 500자 이내로 출력해주세요."
                    ),
                    input_variables=["user_input", "cards"]
                )

                reason_chain = LLMChain(llm=self.model, prompt=tarot_template_reason)
                reason_response = reason_chain.invoke({"input":user_input, "cards":cards})
                answer = reason_response.strip()
        

                return jsonify({"answer": answer})

            except Exception as e:
                return "해석 중 오류가 발생했습니다."
            
        # 일반적인 타로 해석
        def general_reading(self, user_select, cards):
            cards_str = ", ".join(cards)
            try:
                tarot_template_genal = PromptTemplate(
                    template=(
                        "사용자 선택 카테고리: '{user_select}'\n"
                        "타로카드: {cards}\n"
                        "{cards}의 의미는 한줄로 간략히 출력하고, {user_select}를 주제로 종합한 해석은 최대 500자 이내로 출력해주세요.\n"
                    ),
                    input_variables=["user_select", "cards"]
                )

                general_chain = LLMChain(llm=self.model, prompt=tarot_template_genal)
                general_response = general_chain.invoke({"user_select":user_select,"cards":cards_str})
                
                if isinstance(general_response, str):
                    answer = general_response.strip()
                else:
                    answer = general_response
                
                if len(answer) > 500:
                    answer = answer[:500]

                return jsonify({"answer": answer})

            except Exception as e:
                print((f"Error: {e}"))
                return "일반 해석 중 오류가 발생했습니다."
            

    key_path = "c:/Users/SMHRD/api.txt"
    try:
        tarot = TarotBot(api_key_path=key_path)
        if reason:
            response = tarot.interpret_cards(user_input,cards)
        else:
            response = tarot.general_reading(user_select,cards)
        print("response: ",response)
        return response
    
    except Exception as e:
        print("Error",str(e))
        return jsonify({"error":str(e)}),500

    


if __name__ == "__main__":
    app.run(port=7000) #Flask 서버 포트 설정

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:7000
Press CTRL+C to quit
127.0.0.1 - - [06/Sep/2024 16:28:09] "OPTIONS /api/interpret HTTP/1.1" 200 -


recived data:  {'user_select': '오늘의 운세', 'cards': ['Nine of Swords', 'The Hierophant', 'Five of Swords']}


C:\Users\smhrd\AppData\Local\Temp\ipykernel_12996\2691127373.py:69: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  general_chain = LLMChain(llm=self.model, prompt=tarot_template_genal)
127.0.0.1 - - [06/Sep/2024 16:28:12] "POST /api/interpret HTTP/1.1" 200 -


response:  <Response 1143 bytes [200 OK]>
